In [ ]:
import numpy as np
import matplotlib.pylab as plt
import math as m
from pyecharts import Style
from pyecharts import Scatter
from pyecharts import EffectScatter
from pyecharts import Overlap
import socket
import sys
from TransData import UploadParamsService
from TransData.ttypes import *
from thrift.protocol import TBinaryProtocol
from thrift.server import TServer
from thrift.transport import TSocket, TTransport

from TransData import UploadParamsService

__HOST = '127.0.0.1'
__PORT = 7777

#业务代码 
class TransDataHandler(UploadParamsService.Iface):
   
    def __init__(self):
        self.log={}
        self.avr_w1=[]             #类1的均值向量
        self.avr_w2=[]             #类2的均值向量
        self.cov_w1=[]             #类1的协方差
        self.cov_w2=[]             #类2的协方差
        self.correct_w1=[0,0]      #类1的正确分类的数据
        self.correct_w2=[0,0]      #类2的正确分类的数据
        self.error_w1=[0,0]        #类1的错误分类的数据
        self.error_w2=[0,0]        #类2的错误分类的数据
               
    def ping(self):
        print('ping')
        
    def UploadParams(self,params):
        
        #导入训练
        Training_Data=np.loadtxt('D:/杭电与人工智能研究院/光伏教仪/人工智能课程案例/课程实验课件/机器学习课程实验一___贝叶斯估计/课程实验程序/数据/Training_Data.txt')
    
        # Training_Data 800*3  400组正常运行数据  400组阴影遮挡数据 

        #80%训练数据与20%测试数据
        M,N=Training_Data.shape

        #在各组中随机地抽取Num_train个训练样本
        Num_train=int(M//2*0.8)
        temp_w1=np.random.permutation(M//2)
        temp_w2=400+np.random.permutation(M//2)

        #训练集w1和w2各Num_train个样本
        Data_Train_W1=Training_Data[temp_w1[0:Num_train],0:2]
        Data_Train_W2=Training_Data[temp_w2[0:Num_train],0:2]

        #测试集w1和w2各m//2-Num_train个样本
        Data_Test_W1=Training_Data[temp_w1[Num_train:M//2],0:2]
        Data_Test_W2=Training_Data[temp_w2[Num_train:M//2],0:2]
        
        #计算两类训练数据的均值向量
        self.avr_w1=np.mean(Data_Train_W1,axis=0)  #按列求均值
        self.avr_w2=np.mean(Data_Train_W2,axis=0)  #按列求均值
        
        #计算w1类的协方差矩阵
        dim0_w1=Data_Train_W1[:,0]
        dim1_w1=Data_Train_W1[:,1]

        var_01_w1=sum((dim0_w1 - self.avr_w1[0]) * (dim1_w1 - self.avr_w1[1])) / (len(Data_Train_W1) -1)
        avr_dim0_w1=np.var(dim0_w1)
        avr_dim1_w1=np.var(dim1_w1)

        cov_w1=[[avr_dim0_w1, var_01_w1],[var_01_w1, avr_dim1_w1]]

        #计算w2类的协方差矩阵
        dim0_w2=Data_Train_W2[:,0]
        dim1_w2=Data_Train_W2[:,1]

        var_01_w2=sum((dim0_w2 - self.avr_w2[0]) * (dim1_w2 - self.avr_w2[1])) / (len(Data_Train_W2) - 1)

        avr_dim0_w2=np.var(dim0_w2)
        avr_dim1_w2=np.var(dim1_w2)

        cov_w2=[[avr_dim0_w2, var_01_w2],[var_01_w2 ,avr_dim1_w2 ]]

        #转换为矩阵形式
        self.cov_w1=np.mat(cov_w1)
        self.cov_w2=np.mat(cov_w2)
        
        #先验概率
        P_w1=1/2;
        P_w2=1/2;
        
        Num=len(Data_Test_W1)
        
        #合并数据集
        Data_Test=np.vstack(( Data_Test_W1, Data_Test_W2))
       
        self.avr_w1=np.mat(self.avr_w1)
        self.avr_w2=np.mat(self.avr_w2)
        
        data_test=np.mat(Data_Test)

        for i in range(0,Num*2):
            #导入测试数据
            x=Data_Test[i,:]
            #计算测试向量的判决函数
            J1=float(-0.5*(x-self.avr_w1)*self.cov_w1.I*(x-self.avr_w1).T+m.log(P_w1)-0.5*m.log(np.linalg.det(self.cov_w1)))
            J2=float(-0.5*(x-self.avr_w2)*self.cov_w2.I*(x-self.avr_w2).T+m.log(P_w2)-0.5*m.log(np.linalg.det(self.cov_w2)))
            #print(J2)))
            #print(J2)
            #分类
            #类w1测试样本归为类w1
            if i<Num and J1 >= J2:
                self.correct_w1=np.vstack((self.correct_w1,Data_Test[i,:]))
            #类w2测试样本归为类w2
            elif i>=Num and J1 < J2:
                self.correct_w2=np.vstack((self.correct_w2,Data_Test[i,:]))
            #类w1测试样本误判为类w2
            elif (i<Num and J1 < J2):
                self.error_w1=np.vstack((self.error_w1,Data_Test[i,:]))
            #类w2测试样本误判为类w1
            elif (i>=Num and J1 >= J2):
                self.error_w2=np.vstack((self.error_w2,Data_Test[i,:]))

        Result_w1=np.vstack((self.correct_w1,self.error_w2))
        Result_w2=np.vstack((self.correct_w2,self.error_w1))
        #计算分类的正确率与错误率
        Num_correct_w1=len(self.correct_w1)-1
        Num_correct_w2=len(self.correct_w2)-1
        Num_error_w1=len(self.error_w1)-1
        Num_error_w2=len(self.error_w2)-1

        error_rate_w1=Num_error_w1/Num
        error_rate_w2=Num_error_w2/Num
        correct_rate_w1=1-error_rate_w1
        correct_rate_w2=1-error_rate_w2
        return_result=[correct_rate_w1,correct_rate_w2]

        print('w1正确率: %2.2f%%'%(correct_rate_w1*100))
        print('w2正确率: %2.2f%%'%(correct_rate_w2*100))
        print('w1误分到w2: %2.2f%%'%(error_rate_w1*100))
        print('w2误分到w1: %2.2f%%'%(error_rate_w2*100))
        
        #pyecharts 分类结果的可视化
        s_1 = Scatter(title="最小错误率下的贝叶斯决策-测试集",title_text_size=20, title_pos="center",width=1000,height=500)
        s_1.add("Class_1",
                Result_w1[:,0], Result_w1[:,1],
                symbol_size=6,
                xaxis_max=17,
                xaxis_min=12.5,
                xaxis_name='光伏阵列输出电压（V）',
                yaxis_name='光伏阵列输出电流（A）',
                yaxis_name_gap=45,
                legend_pos='center',
                legend_top="8%",
                legend_text_size=14)
        
        s_2 = Scatter(title="最小错误率下的贝叶斯决策-测试集",title_text_size=20, title_pos="center",width=1000,height=500)
        s_2.add("Class_2",
                Result_w2[:,0], Result_w2[:,1],
                symbol_size=6,
                xaxis_max=17,
                xaxis_min=12.5,
                xaxis_name='光伏阵列输出电压（V）',
                yaxis_name='光伏阵列输出电流（A）',
                yaxis_name_gap=45,
                legend_pos='center',
                legend_top="8%",
                legend_text_size=14,)
        
        es_1 = EffectScatter(title="最小错误率下的贝叶斯决策-测试集",title_text_size=20, title_pos="center",width=1000,height=500)
        es_1.add("Class_1错误分类为Class_2",
                 self.error_w1[:,0], self.error_w1[:,1],
                 symbol_size=6,
                 xaxis_max=17,
                 xaxis_min=12.5,
                 xaxis_name='光伏阵列输出电压（V）',
                 yaxis_name='光伏阵列输出电流（A）',
                 yaxis_name_gap=45,
                 legend_pos='center',
                 legend_top="8%",
                 legend_text_size=14)
        
        es_2 = EffectScatter(title="最小错误率下的贝叶斯决策-测试集",title_text_size=20, title_pos="center",width=1000,height=500)
        es_2.add("Class_2错误分类为Class_1",
                 self.error_w2[:,0], self.error_w2[:,1],
                 symbol_size=6,
                 xaxis_max=17,
                 xaxis_min=12.5,
                 xaxis_name='光伏阵列输出电压（V）',
                 yaxis_name='光伏阵列输出电流（A）',
                 yaxis_name_gap=45,
                 legend_pos='center',
                 legend_top="8%",
                 legend_text_size=14)
        
        overlap = Overlap()
        overlap.add(s_1)
        overlap.add(s_2)
        overlap.add(es_1)
        overlap.add(es_2)
        overlap.render()
        
        for key,value in params.items():
            ret = "Key:" + key + "; Value:" + value
            print(ret)
        print("------------------")
        return 'done!!!'
        

def run():
    #服务端业务处理逻辑
    handler = TransDataHandler()
    
    #从Thrift框架转移到业务处理逻辑 
    processor = UploadParamsService.Processor(handler)
    
    #服务器socket  主机地址  端口号
    transport = TSocket.TServerSocket(host=__HOST, port=__PORT)
    
    #传输协议
    tfactory = TTransport.TBufferedTransportFactory()
    
    #通信协议  数据的序列化和反序列化
    pfactory = TBinaryProtocol.TBinaryProtocolFactory()
    
    #服务器类型 --多线程服务器端使用标准的阻塞式 I/O
    server = TServer.TThreadPoolServer(processor, transport, tfactory, pfactory)
    server.setNumThreads(5)  
  

    #启动服务器
    print('Starting the server')
    server.serve()
    
    print('done')
    
if __name__ == '__main__':
    run()
    

Starting the server
w1正确率: 70.00%
w2正确率: 67.50%
w1误分到w2: 30.00%
w2误分到w1: 32.50%
Key:Alice; Value:2341
Key:Beth; Value:9102
Key:Cecil; Value:3258
------------------
